# Task 1 – Boolean Models


## Classic boolean model


Each term is binary (present or not).

Retrieve only documents that exactly satisfy the Boolean expression.

Operators: AND, OR, NOT.

Example query: q = (query AND reformulation) OR (Language AND model)

In [15]:
query = "(query AND reformulation) OR (language AND model)"

In [16]:
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# ---------- 1. Preprocessing ----------
def preprocess(text):
    stemmer = PorterStemmer()
    stops = set(stopwords.words('english'))
    tokens = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    tokens = [stemmer.stem(t) for t in tokens if t not in stops]
    return tokens


# ---------- 2. Load Inverted Index ----------
def load_inverted_index(filepath):
    inverted = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 2:
                term, doc_id = parts[0], parts[1]
                inverted.setdefault(term, set()).add(doc_id)
    return inverted


# ---------- 3. Evaluate Boolean Query ----------
def evaluate_boolean_query(query, inverted_index, all_docs):
    stemmer = PorterStemmer()
    stops = set(stopwords.words('english'))

    # Extract all candidate words (ignoring AND/OR/NOT and parentheses)
    raw_tokens = re.findall(r'\b[a-zA-Z]+\b', query)
    unique_tokens = set(raw_tokens) - {"AND", "OR", "NOT"}

    # Start with the original expression
    expression = query

    # For each token, find its stemmed version and replace it
    for token in unique_tokens:
        if token.lower() in stops:
            continue
        stemmed = stemmer.stem(token.lower())
        docs = inverted_index.get(stemmed, set())
        expression = re.sub(rf'\b{token}\b', f"set({list(docs)})", expression, flags=re.IGNORECASE)

    # Replace logical operators with Python equivalents
    expression = re.sub(r"\bAND\b", "&", expression, flags=re.IGNORECASE)
    expression = re.sub(r"\bOR\b", "|", expression, flags=re.IGNORECASE)
    expression = re.sub(r"\bNOT\b", "all_docs -", expression, flags=re.IGNORECASE)

    # Evaluate expression safely
    try:
        result = eval(expression, {"__builtins__": None}, {"all_docs": all_docs, "set": set})
    except Exception as e:
        print("Error in query:", e)
        print("Expression after replacements:", expression)
        return set()

    return result


# ---------- 4. Example Run ----------
if __name__ == "__main__":
    inverted_index = load_inverted_index("results/inverted_index_weighted.txt")
    all_docs = {f"D{i}" for i in range(1, 7)}

    relevant_docs = evaluate_boolean_query(query, inverted_index, all_docs)

    print("\nClassic Boolean Model Results:")
    print("Query:", query)
    print("Retrieved documents:", sorted(relevant_docs))



Classic Boolean Model Results:
Query: (query AND reformulation) OR (language AND model)
Retrieved documents: ['D1', 'D2', 'D3', 'D4', 'D5', 'D6']


## Fuzzy boolean model


### 🔹 1. Concept Recap

The **Fuzzy Boolean Model** is a hybrid between:

- the **Boolean model** (logical operators `AND`, `OR`, `NOT`)  
- and the **Vector model** (graded, real-valued similarities instead of strict true/false).

Each term weight (e.g., **TF-IDF**) is treated as a **degree of membership** in the interval **[0, 1]**, not binary.

---

### 🔸 Core idea

For each query term \( t \) and document \( d \):

\[
w_{t,d} = \text{TF-IDF weight of term } t \text{ in document } d
\]

Each document’s relevance to a query is computed using **fuzzy logic operators**:

- **AND →** use `min`
- **OR →** use `max`
- **NOT →** use `1 - weight`

---

### 🔹 2. Fuzzy Boolean Operators

| Operator | Boolean | Fuzzy Equivalent | Formula |
|:---------:|:--------:|:----------------:|:--------:|
| **AND** | ∧ | min | \( S_{AND}(d,q) = \min(w_{t1,d}, w_{t2,d}) \) |
| **OR** | ∨ | max | \( S_{OR}(d,q) = \max(w_{t1,d}, w_{t2,d}) \) |
| **NOT** | ¬ | complement | \( S_{NOT}(d,q) = 1 - w_{t,d} \) |

---

✅ **Note:**  
For multi-term queries, you can combine these operators **recursively** to compute the final fuzzy relevance score.


In [ ]:
import os


base_path = r"results/"
inverted_index_file = os.path.join(base_path, "inverted_index_weighted.txt")


inverted_index = {}
all_docs = set()
with open(inverted_index_file, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 4:
            continue
        term, doc, freq, weight = parts[0], parts[1], int(parts[2]), float(parts[3])
        inverted_index.setdefault(term, []).append((doc, freq, weight))
        all_docs.add(doc)
N = len(all_docs)


import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'(?:[A-Za-z]\.)+|[A-Za-z]+(?:[-@]\d+(?:\.\d+)?)|\d+(?:[:\.,\-]\d+)*%?|[A-Za-z]+')
stop_en = set(stopwords.words('english'))
BOOL_OPS = {'AND', 'OR', 'NOT', '(', ')'}


def preprocess_query(qtext: str):
    raw = tokenizer.tokenize(qtext.lower())
    out = []
    for tok in raw:
        up = tok.upper()
        if up in BOOL_OPS:
            out.append(up)
        else:
            if tok in stop_en:
                continue
            out.append(ps.stem(tok))
    return out


PRECEDENCE = {'NOT': 3, 'AND': 2, 'OR': 1}
def to_rpn(tokens):
    output = []
    stack = []
    for t in tokens:
        if t in BOOL_OPS and t not in {'(', ')'}:
            while stack and stack[-1] in PRECEDENCE and PRECEDENCE[stack[-1]] >= PRECEDENCE[t]:
                output.append(stack.pop())
            stack.append(t)
        elif t == '(':
            stack.append(t)
        elif t == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            if stack and stack[-1] == '(':
                stack.pop()
        else:
            output.append(t)
    while stack:
        output.append(stack.pop())
    return output


def postings_set(term): # returns set of documents containing the term
    return set(d for d, _, _ in inverted_index.get(term, []))


def term_score_vector(term): # returns dict of {doc: weight} for the term
   
    docs = inverted_index.get(term, [])
    if not docs:
        return {}
    return {d: w for d, _, w in docs}



def eval_classic_rpn(rpn):
    stack = []
    for t in rpn:
        if t == 'NOT':
            a = stack.pop() if stack else set()
            stack.append(all_docs - a)
        elif t == 'AND':
            b = stack.pop() if stack else set()
            a = stack.pop() if stack else set()
            stack.append(a & b)
        elif t == 'OR':
            b = stack.pop() if stack else set()
            a = stack.pop() if stack else set()
            stack.append(a | b)
        else:
            stack.append(postings_set(t))
    return stack.pop() if stack else set()


def eval_fuzzy_rpn(rpn):
    stack = []
    for t in rpn:
        if t == 'NOT':
            a = stack.pop() if stack else {}
            stack.append({d: 1.0 - a.get(d, 0.0) for d in all_docs})
        elif t == 'AND':
            b = stack.pop() if stack else {}
            a = stack.pop() if stack else {}
            stack.append({d: min(a.get(d, 0.0), b.get(d, 0.0)) for d in all_docs})
        elif t == 'OR':
            b = stack.pop() if stack else {}
            a = stack.pop() if stack else {}
            stack.append({d: max(a.get(d, 0.0), b.get(d, 0.0)) for d in all_docs})
        else:
            stack.append(term_score_vector(t))
    return stack.pop() if stack else {}


def run_classic_verbose(query_text):
    toks = preprocess_query(query_text)
    print("\n[Classic] Requête :", query_text)
    print("[Classic] Tokens  :", toks)
    rpn = to_rpn(toks)
    print("[Classic] RPN     :", rpn)

    stack = []
    steps = []
    for t in rpn:
        if t == 'NOT':
            a = stack.pop() if stack else set()
            result = all_docs - a
            steps.append(f"NOT -> Complément de {a}\n= {result}")
            stack.append(result)
        elif t == 'AND':
            b = stack.pop() if stack else set()
            a = stack.pop() if stack else set()
            result = a & b
            steps.append(f"AND -> {a} ∩ {b}\n= {result}")
            stack.append(result)
        elif t == 'OR':
            b = stack.pop() if stack else set()
            a = stack.pop() if stack else set()
            result = a | b
            steps.append(f"OR  -> {a} ∪ {b}\n= {result}")
            stack.append(result)
        else:
            docs = postings_set(t)
            steps.append(f"Terme '{t}' -> docs {docs}")
            stack.append(docs)

    print('\n[Classic] Détail des étapes:')
    for i, s in enumerate(steps, 1):
        print(f"- Étape {i}: {s}")
    result_docs = sorted(list(stack.pop() if stack else set()))
    print('\n[Classic] Résultat final (docs):')
    print(result_docs)
    return result_docs

# Prints:
# Query
# Tokens
# RPN
# Each evaluation step (AND, OR, NOT)

# Final list of documents
def run_fuzzy_verbose(query_text, topk=20):
    toks = preprocess_query(query_text)
    print("\n[Fuzzy] Requête :", query_text)
    print("[Fuzzy] Tokens  :", toks)
    rpn = to_rpn(toks)
    print("[Fuzzy] RPN     :", rpn)

    stack = []
    steps = []
    for t in rpn:
        if t == 'NOT':
            a = stack.pop() if stack else {}
            result = {d: 1.0 - a.get(d, 0.0) for d in all_docs}
            steps.append("NOT -> 1 - score")
            stack.append(result)
        elif t == 'AND':
            b = stack.pop() if stack else {}
            a = stack.pop() if stack else {}
            result = {d: min(a.get(d, 0.0), b.get(d, 0.0)) for d in all_docs}
            steps.append("AND -> min(scores)")
            stack.append(result)
        elif t == 'OR':
            b = stack.pop() if stack else {}
            a = stack.pop() if stack else {}
            result = {d: max(a.get(d, 0.0), b.get(d, 0.0)) for d in all_docs}
            steps.append("OR  -> max(scores)")
            stack.append(result)
        else:
            vec = term_score_vector(t)
            steps.append(f"Terme '{t}' -> RSV(d, t) (normalisé), {len(vec)} docs")
            stack.append(vec)

    print('\n[Fuzzy] Détail des étapes:')
    for i, s in enumerate(steps, 1):
        print(f"- Étape {i}: {s}")

    scores = stack.pop() if stack else {}
    ranking = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    print("\n[Fuzzy] Top documents (doc, RSV):")
    for d, s in ranking[:topk]:
        print(f"{d}\t{s:.4f}")
    return ranking


query = "(query AND reformulation) OR (language AND model)"
run_classic_verbose(query)
run_fuzzy_verbose(query, topk=20)



[Classic] Requête : (query AND reformulation) OR (language AND model)
[Classic] Tokens  : ['queri', 'AND', 'reformul', 'OR', 'languag', 'AND', 'model']
[Classic] RPN     : ['queri', 'reformul', 'AND', 'languag', 'model', 'AND', 'OR']

[Classic] Détail des étapes:
- Étape 1: Terme 'queri' -> docs {'D2', 'D5', 'D4', 'D1'}
- Étape 2: Terme 'reformul' -> docs {'D4', 'D1'}
- Étape 3: AND -> {'D2', 'D5', 'D4', 'D1'} ∩ {'D4', 'D1'}
= {'D4', 'D1'}
- Étape 4: Terme 'languag' -> docs {'D3', 'D1', 'D2', 'D6', 'D4', 'D5'}
- Étape 5: Terme 'model' -> docs {'D3', 'D1', 'D2', 'D6', 'D4', 'D5'}
- Étape 6: AND -> {'D3', 'D1', 'D2', 'D6', 'D4', 'D5'} ∩ {'D3', 'D1', 'D2', 'D6', 'D4', 'D5'}
= {'D3', 'D2', 'D6', 'D4', 'D5', 'D1'}
- Étape 7: OR  -> {'D4', 'D1'} ∪ {'D3', 'D2', 'D6', 'D4', 'D5', 'D1'}
= {'D3', 'D2', 'D6', 'D4', 'D5', 'D1'}

[Classic] Résultat final (docs):
['D1', 'D2', 'D3', 'D4', 'D5', 'D6']

[Fuzzy] Requête : (query AND reformulation) OR (language AND model)
[Fuzzy] Tokens  : ['queri', 'AN

[('D4', 0.267582),
 ('D1', 0.238764),
 ('D5', 0.100343),
 ('D2', 0.050172),
 ('D3', 0.037629),
 ('D6', 0.037629)]

# Extended Boolean Model

## 🔹 1. Concept Recap

The **Extended Boolean Model** replaces the strict Boolean **AND / OR** logic with continuous functions that measure **how much a document satisfies a query**.

It uses the **p-norm operator**, where the parameter \( p \) controls how strict or relaxed the matching is:

| **p value** | **Behavior** |
|:------------:|:-------------|
| \( p \to \infty \) | strict Boolean (perfect AND/OR) |
| \( p = 1 \) | loose, soft matching (closer to vector model) |
| **Typical value** | between 2 and 5 |

---

## 🧩 Formulas

Let \( w_{d_i} \) be the **TF–IDF weight** of term *i* in document *d*, normalized to \([0, 1]\).

---

### 🔸 AND Query

\[
S_{\text{AND}}(d, q) = 
\left( \frac{1}{n} \sum_{i=1}^{n} (w_{d_i})^p \right)^{\frac{1}{p}}
\]

---

### 🔸 OR Query

\[
S_{\text{OR}}(d, q) = 
1 - 
\left( 
\frac{1}{n} 
\sum_{i=1}^{n} (1 - w_{d_i})^p 
\right)^{\frac{1}{p}}
\]

---

where  

- \( n \) = number of terms in the query  
- \( p \) = the **p-norm parameter** controlling the **strictness of matching**

---

✅ **Intuition:**

- When \( p \) is large → behavior approaches **strict Boolean logic**  
- When \( p \) is small → matching becomes **more flexible**, similar to **vector-space similarity**


In [4]:
import pandas as pd
import numpy as np
from collections import defaultdict

# ------------------------------------------------------------
# 1. LOAD THE INVERTED INDEX
# ------------------------------------------------------------
inverted_path = "results/inverted_index_weighted.txt"

data = pd.read_csv(inverted_path, sep="\t", header=None, names=["term", "doc", "freq", "tfidf"])

# Build structure: {doc: {term: tfidf}}
doc_dict = defaultdict(dict)
for _, row in data.iterrows():
    doc_dict[row["doc"]][str(row["term"]).lower()] = float(row["tfidf"])

# ------------------------------------------------------------
# 2. EXTENDED BOOLEAN MODEL FUNCTIONS
# ------------------------------------------------------------
def extended_boolean_score(doc_weights, query_terms, operator="AND", p=2):
    """
    Compute the Extended Boolean model score for one document.
    - doc_weights: dict {term: weight}
    - query_terms: list of query tokens (strings)
    - operator: 'AND' or 'OR'
    - p: float, p-norm parameter
    """
    # Extract weights for query terms (default 0 if term not in doc)
    w = np.array([doc_weights.get(term.lower(), 0.0) for term in query_terms])
    n = len(w)
    if n == 0:
        return 0.0

    if operator.upper() == "AND":
        return (np.sum(w ** p) / n) ** (1 / p)
    elif operator.upper() == "OR":
        return 1 - ((np.sum((1 - w) ** p) / n) ** (1 / p))
    else:
        raise ValueError("Operator must be 'AND' or 'OR'")

# ------------------------------------------------------------
# 3. QUERY EXECUTION
# ------------------------------------------------------------
def extended_boolean_search(query, p=2):
    """
    Execute an AND/OR query across all documents using the Extended Boolean Model.
    """
    # Detect operator
    query_lower = query.lower()
    if " and " in query_lower:
        operator = "AND"
        terms = [t.strip() for t in query_lower.split("and")]
    elif " or " in query_lower:
        operator = "OR"
        terms = [t.strip() for t in query_lower.split("or")]
    else:
        operator = "AND"
        terms = [query_lower.strip()]
    
    # Compute scores for all docs
    results = {}
    for doc, weights in doc_dict.items():
        score = extended_boolean_score(weights, terms, operator=operator, p=p)
        results[doc] = round(score, 4)

    # Sort results by descending score
    results = dict(sorted(results.items(), key=lambda x: x[1], reverse=True))
    return results

# ------------------------------------------------------------
# 4. TEST
# ------------------------------------------------------------
query = "10% AND 12%"
p = 2  # try 1, 2, or higher for stricter matching

results = extended_boolean_search(query, p=p)

print(f"\n🔍 Extended Boolean Query: {query} (p={p})\n")
print("Doc\tScore")
for doc, score in results.items():
    print(f"{doc}\t{score}")



🔍 Extended Boolean Query: 10% AND 12% (p=2)

Doc	Score
D2	0.1992
D4	0.0664
D6	0.0
D1	0.0
D5	0.0
D3	0.0


In [5]:
import math

def eval_extended_rpn(rpn, p=2):
    """
    Evaluate a query in Reverse Polish Notation using the Extended Boolean Model.
    p controls strictness: p=1 soft, p→∞ strict boolean
    """
    stack = []
    for t in rpn:
        if t == 'NOT':
            a = stack.pop() if stack else {}
            # Negation: invert membership degree
            result = {d: 1 - a.get(d, 0.0) for d in all_docs}
            stack.append(result)

        elif t == 'AND':
            b = stack.pop() if stack else {}
            a = stack.pop() if stack else {}
            result = {}
            for d in all_docs:
                # average of powered weights (p-norm)
                w = ((a.get(d, 0.0)**p + b.get(d, 0.0)**p) / 2)**(1/p)
                result[d] = w
            stack.append(result)

        elif t == 'OR':
            b = stack.pop() if stack else {}
            a = stack.pop() if stack else {}
            result = {}
            for d in all_docs:
                # OR p-norm version
                w = 1 - (( (1 - a.get(d, 0.0))**p + (1 - b.get(d, 0.0))**p ) / 2)**(1/p)
                result[d] = w
            stack.append(result)

        else:
            # term → weight vector from index
            stack.append(term_score_vector(t))

    return stack.pop() if stack else {}

query_text = "query AND language"
toks = preprocess_query(query_text)
rpn = to_rpn(toks)
scores = eval_extended_rpn(rpn, p=2)

for d, s in sorted(scores.items()):
    print(f"{d}: {s:.4f}")


D1: 0.1741
D2: 0.0588
D3: 0.0266
D4: 0.2824
D5: 0.0851
D6: 0.0532
